In [56]:
from astroquery.gaia import Gaia
import pandas as pd
from astropy.table import Table

In [23]:
from astropy.io import ascii
data = ascii.read('brokenTable.csv')
data

source_id,ra,dec,parallax,parallax_error,parallax_over_error,pmra,pmra_error,pmdec,pmdec_error,ruwe,target_id,target_ra,target_dec,target_parallax,target_pmra,target_pmdec
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str27,float64,float64,float64,float64,float64
4.07e+18,271.349628,-25.60956332,6.258289544,1.232649,5.077105927,-1.889637979,1.3439337,2.722026127,1.0029136,1.536287,GaiaDR3 4065645218118617600,271.349628,-25.60956332,6.258289544,-1.889637979,2.722026127
1.82e+18,297.2798818,16.00609245,5.891184478,1.0943899,5.383076432,-5.492079429,0.9150198,1.425150489,0.8917758,1.8236009,GaiaDR3 1819862241336726528,297.2798818,16.00609245,5.891184478,-5.492079429,1.425150489


In [42]:
job = Gaia.launch_job_async("""
SELECT g3.source_id, g3.designation, g3.ra, g3.dec, g3.parallax, g3.parallax_error, g3.parallax_over_error,
g3.pmra, g3.pmra_error, g3.pmdec, g3.pmdec_error, g3.phot_g_mean_mag, g3.bp_rp,
0.01745* (distance(POINT('ICRS', t1.ra, t1.dec), POINT('ICRS', g3.ra, g3.dec)))/t1.parallax AS variable

FROM gaiadr3.gaia_source AS g3, tap_upload.table_test_w AS t1

WHERE 1 = contains(
   POINT(t1.ra, t1.dec),
   CIRCLE(g3.ra, g3.dec, 0.0114591)) AND
   
   t1.source_id != g3.source_id AND
   
abs(1/t1.parallax - 1/g3.parallax) -
  2*0.01745*distance(POINT('ICRS', t1.ra,t1.dec),
  POINT('ICRS', g3.ra, g3.dec))/t1.parallax
  < 3*sqrt(power(t1.parallax_error,2)/power(t1.parallax, 4)
   + power(g3.parallax_error, 2)/power(g3.parallax, 4)) AND
   
   
   sqrt(power((t1.pmra - g3.pmra), 2) +power((t1.pmdec - g3.pmdec), 2))
  - (7.42e-3 * power(t1.parallax, 1.5) *
  power(distance(POINT('ICRS', t1.ra, t1.dec), POINT('ICRS', g3.ra, g3.dec)), -0.5))
  < 3*sqrt(((power((t1.pmra_error), 2) + power((g3.pmra_error), 2))
  * power((t1.pmra - g3.pmra), 2)
  + (power((t1.pmdec_error), 2) + power((g3.pmdec_error), 2))
   * power((t1.pmdec - g3.pmdec), 2))/
  (power((t1.pmra - g3.pmra), 2) + power((t1.pmdec - g3.pmdec), 2))) AND
 
sqrt(((power((t1.pmra_error), 2) +power((g3.pmra_error), 2))
  * power((t1.pmra - g3.pmra), 2)
  + (power((t1.pmdec_error), 2) + power((g3.pmdec_error), 2))
  * power((t1.pmdec -g3.pmdec), 2))
  /(power((t1.pmra - g3.pmra),2) +  power((t1.pmdec - g3.pmdec), 2))) < 1.5
 
""", upload_resource=data, upload_table_name="table_test_w", verbose=True)

Launched query: '
SELECT g3.source_id, g3.designation, g3.ra, g3.dec, g3.parallax, g3.parallax_error, g3.parallax_over_error,
g3.pmra, g3.pmra_error, g3.pmdec, g3.pmdec_error, g3.phot_g_mean_mag, g3.bp_rp,
0.01745* (distance(POINT('ICRS', t1.ra, t1.dec), POINT('ICRS', g3.ra, g3.dec)))/t1.parallax AS variable

FROM gaiadr3.gaia_source AS g3, tap_upload.table_test_w AS t1

WHERE 1 = contains(
   POINT(t1.ra, t1.dec),
   CIRCLE(g3.ra, g3.dec, 0.0114591)) AND
   
   t1.source_id != g3.source_id AND
   
abs(1/t1.parallax - 1/g3.parallax) -
  2*0.01745*distance(POINT('ICRS', t1.ra,t1.dec),
  POINT('ICRS', g3.ra, g3.dec))/t1.parallax
  < 3*sqrt(power(t1.parallax_error,2)/power(t1.parallax, 4)
   + power(g3.parallax_error, 2)/power(g3.parallax, 4)) AND
   
   
   sqrt(power((t1.pmra - g3.pmra), 2) +power((t1.pmdec - g3.pmdec), 2))
  - (7.42e-3 * power(t1.parallax, 1.5) *
  power(distance(POINT('ICRS', t1.ra, t1.dec), POINT('ICRS', g3.ra, g3.dec)), -0.5))
  < 3*sqrt(((power((t1.pmra_error), 2) 

INFO:astropy:Query finished.


INFO: Query finished. [astroquery.utils.tap.core]


In [43]:
results = job.get_results()

In [44]:
results

source_id,DESIGNATION,ra,dec,parallax,parallax_error,parallax_over_error,pmra,pmra_error,pmdec,pmdec_error,phot_g_mean_mag,bp_rp,variable
,,deg,deg,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,mag,mag,
int64,object,float64,float64,float64,float32,float32,float64,float32,float64,float32,float32,float32,float64
4065645183757221120,Gaia DR3 4065645183757221120,271.33883777774344,-25.615146111509237,0.5443373624987456,1.2268751,0.4436779,-1.1388340684549505,1.0604534,-0.17750020704266165,0.6549209,19.879362,1.9039154,3.1278711123986466e-05
4065645183759399424,Gaia DR3 4065645183759399424,271.3395723097803,-25.614793128149604,-0.40129303667355737,0.77464455,-0.51803505,0.8141644929150853,0.79366165,6.320781091648299,0.53075004,19.953459,2.6819363,2.918710266532974e-05
4065645218118621056,Gaia DR3 4065645218118621056,271.3434075514195,-25.610645829085836,0.41172142481808305,0.39619777,1.0391816,-6.52107810374375,0.44227242,1.390204513036163,0.2920033,19.277206,--,1.5929086475871104e-05
4065645213819997824,Gaia DR3 4065645213819997824,271.3448052915299,-25.613396822935467,0.19089443804972386,0.14222185,1.34223,-1.6186043877321397,0.1675245,0.9415703261126149,0.12099118,17.950872,2.9635773,1.6164530927324616e-05
4065645218116842112,Gaia DR3 4065645218116842112,271.35784244910565,-25.616670149295246,0.8127407809465433,0.28927842,2.8095453,-0.9982699680450702,0.32693353,0.8950522428080296,0.24499618,18.977167,1.5210304,2.8622470837020114e-05
4065645218116849408,Gaia DR3 4065645218116849408,271.3603072027157,-25.613152514673697,-0.37761639575734374,0.36190408,-1.0434157,1.209881389711271,0.3764282,0.013424489811979369,0.26023135,19.255398,1.8505096,2.8655551895856604e-05
4065645218116851584,Gaia DR3 4065645218116851584,271.3515308483659,-25.620577714823508,-0.18231135347395858,0.47854817,-0.38096762,1.6646777240483832,0.65328294,0.7677951773186285,0.49747333,18.59742,2.4617863,3.108187195679221e-05
4065645218116856576,Gaia DR3 4065645218116856576,271.3554546608691,-25.61652908661444,0.41038480751990175,0.3896502,1.0532134,-0.04128521263045212,0.50062525,-1.2465158163123815,0.33230504,19.285336,2.2806053,2.43282263623814e-05


In [37]:
for el in results:
    print(el)

     source_id              DESIGNATION                  ra                dec              parallax     parallax_error parallax_over_error        pmra       pmra_error       pmdec        pmdec_error phot_g_mean_mag bp_rp        variable      
                                                        deg                deg                mas             mas                                mas / yr      mas / yr       mas / yr        mas / yr        mag        mag                       
------------------- ---------------------------- ----------------- ------------------- ----------------- -------------- ------------------- ----------------- ---------- ------------------ ----------- --------------- ----- ---------------------
4065644457897700608 Gaia DR3 4065644457897700608 271.3172130521475 -25.666861815322243 0.709701846465265      0.4615936           1.5375036 -0.63257734180553  0.6683343 0.7974079227247037   0.4191712       18.414875    -- 0.0001793450540303094
     source_id          

     source_id              DESIGNATION                  ra                dec              parallax     parallax_error parallax_over_error        pmra        pmra_error       pmdec        pmdec_error phot_g_mean_mag bp_rp        variable       
                                                        deg                deg                mas             mas                                mas / yr       mas / yr       mas / yr        mas / yr        mag        mag                        
------------------- ---------------------------- ----------------- ------------------- ----------------- -------------- ------------------- ------------------ ---------- ------------------ ----------- --------------- ----- ----------------------
4065651024916777728 Gaia DR3 4065651024916777728 271.3970670047075 -25.591701914913763 0.709706879616114     0.27899662            2.543783 1.5097542841660803 0.35483277 1.1387550053476316  0.27706873       18.957054    -- 0.00012926758137266517
     source_id  

     source_id              DESIGNATION                  ra                 dec               parallax       parallax_error parallax_over_error        pmra        pmra_error       pmdec        pmdec_error phot_g_mean_mag   bp_rp         variable      
                                                        deg                 deg                 mas               mas                                mas / yr       mas / yr       mas / yr        mas / yr        mag          mag                        
------------------- ---------------------------- ------------------ ------------------- -------------------- -------------- ------------------- ------------------ ---------- ------------------ ----------- --------------- --------- --------------------
4065651231075618560 Gaia DR3 4065651231075618560 271.38528198330476 -25.578280117907237 -0.22781479184736864     0.23635684         -0.96385956 -3.267881976503889  0.3033036 1.2982021431850839   0.2258565       18.598413 2.3895035 0.00012508939

4065652090069744896 Gaia DR3 4065652090069744896 271.3635305825497 -25.55014251465547 0.13681598924235683     0.55583745          0.24614388 -0.07817626722094231 0.50782526 0.5381765531380562  0.33969697         19.6746    -- 0.0001693323881907803
     source_id              DESIGNATION                  ra                 dec              parallax     parallax_error parallax_over_error        pmra        pmra_error        pmdec         pmdec_error phot_g_mean_mag   bp_rp          variable       
                                                        deg                 deg                mas             mas                                mas / yr       mas / yr        mas / yr         mas / yr        mag          mag                          
------------------- ---------------------------- ------------------ ------------------- ----------------- -------------- ------------------- ------------------ ---------- -------------------- ----------- --------------- --------- ----------------

     source_id              DESIGNATION                  ra                dec               parallax      parallax_error parallax_over_error        pmra        pmra_error        pmdec        pmdec_error phot_g_mean_mag   bp_rp          variable       
                                                        deg                deg                 mas              mas                                mas / yr       mas / yr        mas / yr        mas / yr        mag          mag                          
------------------- ---------------------------- ------------------ ------------------ ------------------- -------------- ------------------- ------------------ ---------- ------------------- ----------- --------------- --------- ----------------------
4065639205161340160 Gaia DR3 4065639205161340160 271.41437958724094 -25.61171543780858 0.19332000728310347     0.37468994           0.5159466 1.0535006466475008   0.464511 0.38052457914546334  0.28332654       19.420994 1.8343868 0.000162919

...

     source_id              DESIGNATION                  ra                dec               parallax      parallax_error parallax_over_error        pmra        pmra_error      pmdec       pmdec_error phot_g_mean_mag bp_rp        variable       
                                                        deg                deg                 mas              mas                                mas / yr       mas / yr      mas / yr       mas / yr        mag        mag                        
------------------- ---------------------------- ------------------ ------------------ ------------------- -------------- ------------------- ------------------ ---------- ---------------- ----------- --------------- ----- ----------------------
4065646455067955200 Gaia DR3 4065646455067955200 271.31951409340775 -25.55206682704155 0.24879794180644985     0.20940961           1.1880922 0.8516734770550728 0.28725436 1.65786891824586  0.22058405       18.786535    -- 0.00017730670328365553
     source_id  

     source_id              DESIGNATION                  ra                 dec              parallax      parallax_error parallax_over_error        pmra        pmra_error        pmdec        pmdec_error phot_g_mean_mag bp_rp       variable      
                                                        deg                 deg                mas              mas                                mas / yr       mas / yr        mas / yr        mas / yr        mag        mag                      
------------------- ---------------------------- ------------------ ------------------- ------------------ -------------- ------------------- ------------------ ---------- ------------------- ----------- --------------- ----- --------------------
4065632779882770048 Gaia DR3 4065632779882770048 271.35978560009335 -25.672759170364074 0.2802646888786975     0.22955772           1.2208898 -1.908107172978793  0.2971974 -0.2861138400255718  0.22059813       18.211718    -- 0.000178049425819038
     source_

     source_id              DESIGNATION                  ra                 dec              parallax      parallax_error parallax_over_error         pmra        pmra_error       pmdec        pmdec_error phot_g_mean_mag bp_rp        variable       
                                                        deg                 deg                mas              mas                                 mas / yr       mas / yr       mas / yr        mas / yr        mag        mag                        
------------------- ---------------------------- ------------------ ------------------- ------------------ -------------- ------------------- ------------------- ---------- ------------------ ----------- --------------- ----- ----------------------
4065633119203694464 Gaia DR3 4065633119203694464 271.39740130477355 -25.661070728292234 0.8760089204595304      0.4061622           2.1567957 -1.7576351272527855  0.5268716 2.0926891808391637   0.3542949        19.34306    -- 0.00018721335351874928
    

     source_id              DESIGNATION                  ra                dec               parallax      parallax_error parallax_over_error        pmra        pmra_error        pmdec         pmdec_error phot_g_mean_mag   bp_rp          variable       
                                                        deg                deg                 mas              mas                                mas / yr       mas / yr        mas / yr         mas / yr        mag          mag                          
------------------- ---------------------------- ----------------- ------------------- ------------------- -------------- ------------------- ------------------ ---------- -------------------- ----------- --------------- --------- ----------------------
4065633187940826880 Gaia DR3 4065633187940826880 271.3778128909042 -25.642309864040776 -0.3721835842906827      0.6637827          -0.5607009 0.9206608209717032  0.6377551 -0.43864414444826066  0.43785438       19.415112 2.0122204 0.00011

4065633226559299072 Gaia DR3 4065633226559299072 271.4117548939776 -25.64011312401203 -0.10166675604113187     0.35775575         -0.28417924 -4.722680823907291  0.5258227 1.5852154849843132   0.3588526           18.42 2.05694 0.00017790870313730194
     source_id              DESIGNATION                  ra                dec              parallax      parallax_error parallax_over_error        pmra        pmra_error       pmdec        pmdec_error phot_g_mean_mag bp_rp        variable       
                                                        deg                deg                mas              mas                                mas / yr       mas / yr       mas / yr        mas / yr        mag        mag                        
------------------- ---------------------------- ----------------- ------------------- ------------------ -------------- ------------------- ------------------ ---------- ------------------ ----------- --------------- ----- ----------------------
406563322

In [30]:
query1 = """SELECT g3.source_id, g3.ra, g3.dec WHERE g3.source_id = 4065644457897700608
FROM gaiadr3.gaia_source AS g3
"""

In [63]:
job2 = Gaia.launch_job_async("""
SELECT g3.source_id, g3.designation, g3.ra, g3.dec, g3.parallax, g3.parallax_error, g3.parallax_over_error,
g3.pmra, g3.pmra_error, g3.pmdec, g3.pmdec_error, g3.phot_g_mean_mag, g3.bp_rp,
0.0114591*t1.parallax AS variable

FROM gaiadr3.gaia_source AS g3, tap_upload.table_test_w AS t1

WHERE 1 = contains(
   POINT(t1.ra, t1.dec),
   CIRCLE(g3.ra, g3.dec, 0.0114591*t1.parallax))
   
   AND t1.source_id != g3.source_id
   
""", upload_resource=data, upload_table_name="table_test_w", verbose=True)
job2.get_results()

Launched query: '
SELECT g3.source_id, g3.designation, g3.ra, g3.dec, g3.parallax, g3.parallax_error, g3.parallax_over_error,
g3.pmra, g3.pmra_error, g3.pmdec, g3.pmdec_error, g3.phot_g_mean_mag, g3.bp_rp,
0.0114591*t1.parallax AS variable

FROM gaiadr3.gaia_source AS g3, tap_upload.table_test_w AS t1

WHERE 1 = contains(
   POINT(t1.ra, t1.dec),
   CIRCLE(g3.ra, g3.dec, 0.0114591*t1.parallax))
   
   AND t1.source_id != g3.source_id
   
'
------>https
host = gea.esac.esa.int:443
context = /tap-server/tap/async
Content-type = multipart/form-data; boundary====1678726709251===
303 303
[('Date', 'Mon, 13 Mar 2023 16:58:29 GMT'), ('Server', 'Apache/2.4.6 (CentOS) OpenSSL/1.0.2k-fips mod_jk/1.2.43'), ('Cache-Control', 'no-cache, no-store, max-age=0, must-revalidate'), ('Pragma', 'no-cache'), ('Expires', '0'), ('X-XSS-Protection', '1; mode=block'), ('X-Frame-Options', 'SAMEORIGIN'), ('X-Content-Type-Options', 'nosniff'), ('Set-Cookie', 'JSESSIONID=307D6C2C95FA55EE6F2AADDECAB6C0A1; Path=/tap-

INFO:astropy:Query finished.


INFO: Query finished. [astroquery.utils.tap.core]


source_id,DESIGNATION,ra,dec,parallax,parallax_error,parallax_over_error,pmra,pmra_error,pmdec,pmdec_error,phot_g_mean_mag,bp_rp,variable
,,deg,deg,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,mag,mag,
int64,object,float64,float64,float64,float32,float32,float64,float32,float64,float32,float32,float32,float64
4065644526654191744,Gaia DR3 4065644526654191744,271.3519796847838,-25.653984696187347,-6.415684550640108,0.9950957,-6.4473042,-0.8473619653319404,1.1259881,-5.522698967374573,0.6421049,19.795065,--,0.0717143657136504
4065644393482963328,Gaia DR3 4065644393482963328,271.32282832289604,-25.672554092980548,-0.4872580181415342,0.3244829,-1.5016448,-0.6548669322465617,0.40346688,-5.891718018298503,0.30061093,18.110996,--,0.0717143657136504
4065644457897700608,Gaia DR3 4065644457897700608,271.3172130521475,-25.666861815322243,0.709701846465265,0.4615936,1.5375036,-0.63257734180553,0.6683343,0.7974079227247037,0.4191712,18.414875,--,0.0717143657136504
4065644492293079040,Gaia DR3 4065644492293079040,271.3405752660384,-25.65843232213764,0.2076430752352334,0.07757511,2.6766713,-4.402674789058295,0.11563106,-2.29497148192742,0.08805446,16.968048,--,0.0717143657136504
4065644496562128896,Gaia DR3 4065644496562128896,271.3353440078027,-25.66905700785701,1.6503944688380636,0.7679752,2.1490204,-4.3610974804506855,0.96268713,-9.644479126674216,0.7118818,19.833523,--,0.0717143657136504
4065644530996974464,Gaia DR3 4065644530996974464,271.34563355720786,-25.65661858087744,0.5048040053817896,0.2749941,1.8356904,3.1835658703928575,0.35702258,-6.051785528375276,0.27028263,18.823042,--,0.0717143657136504
4065644634001351168,Gaia DR3 4065644634001351168,271.2978690455147,-25.66043951881445,0.3791126459071717,1.1998138,0.3159762,-5.960907053525721,1.6560599,-0.8831071595044854,1.284148,20.171684,--,0.0717143657136504
4065644664075604352,Gaia DR3 4065644664075604352,271.30008028608484,-25.653372591508678,0.8064762145061742,0.6851046,1.1771578,0.7451214807542916,0.86005366,-3.4646737706740947,0.54108506,19.426498,--,0.0717143657136504


In [64]:
results = job2.get_results()


In [65]:
for el in results['source_id']:
    print(el)

4065644526654191744
4065644393482963328
4065644457897700608
4065644492293079040
4065644496562128896
4065644530996974464
4065644634001351168
4065644664075604352
4065644664075621248
4065644801531985408
4065644801532087040
4065644801532088832
4065644805895413632
4065644977598582400
4065645046317975040
4065644496565953536
4065644496565953792
4065644496637192576
4065644496637194240
4065644496637196032
4065644530921828352
4065644634076317440
4065644634076317952
4065644943238715520
4065644943238715648
4065644943238716416
4065644943238721152
4065644943238733568
4065644943238735360
4065651231078937088
4065651265507972480
4065650951888547840
4065650990556617600
4065651024916698624
4065651059348769408
4065651093635922176
4065650956200938880
4065651231078937344
4065651231078939776
4065651231078941824
4065651231078948992
4065651231078956800
4065651231078966656
4065651265435364352
4065650951888489088
4065650951888538368
4065650951888538624
4065650951888538752
4065650951888539136
4065650951888547712


4065651231075480192
4065651231075480704
4065651231075486464
4065651231075491968
4065651231075494144
4065651231075495424
4065651231075497984
4065651231075501056
4065651231075507840
4065651231075511424
4065651231075511552
4065651231075511680
4065651231075513472
4065651231075513600
4065651231075515520
4065651231075523840
4065651231075526016
4065651231075528576
4065651231075528832
4065651231075528960
4065651231075530368
4065651231075530496
4065651231075533568
4065651231075535104
4065651231075535232
4065651231075537664
4065651231075543680
4065651231075545728
4065651231075546112
4065651231075548544
4065651231075557504
4065651231075557632
4065651231075565056
4065651231075565952
4065651231075568768
4065651231075569536
4065651231075572736
4065651231075578880
4065651231075579264
4065651231075579392
4065651231075579520
4065651231075581184
4065651231075581440
4065651231075585408
4065651231075587200
4065651231075592064
4065651231075595136
4065651231075599616
4065651231075599744
4065651231075599872


4065652090069558400
4065652090069562112
4065652090069563392
4065652090069568000
4065652090069578624
4065652090069585280
4065652090069585408
4065652090069585792
4065652090069593600
4065652090069596800
4065652090069596928
4065652090069597056
4065652090069597184
4065652090069597312
4065652090069597440
4065652090069599616
4065652090069599872
4065652090069600000
4065652090069600128
4065652090069604224
4065652090069604736
4065652090069604864
4065652090069604992
4065652090069621376
4065652090069627008
4065652090069627136
4065652090069628800
4065652090069630976
4065652090069639680
4065652090069641216
4065652090069648768
4065652090069657856
4065652090069661952
4065652090069662208
4065652090069662464
4065652090069680896
4065652090069681536
4065652090069682816
4065652090069682944
4065652090069683072
4065652090069683584
4065652090069688064
4065652090069693568
4065652090069699584
4065652090069701376
4065652090069702784
4065652090069708288
4065652090069720320
4065652090069720960
4065652090069721472


4065644393558743808
4065644393558857600
4065644393558894592
4065644393558974720
4065644393558978304
4065644393559109248
4065644393559325952
4065644393578386176
4065644393578386304
4065644393578409216
4065644393578409600
4065644393578410112
4065644393578421760
4065644393578427392
4065644393578427520
4065644423537940608
4065644492293159808
4065644423537940736
4065644423537942272
4065644423557452800
4065644457897698944
4065644423573367424
4065644423573376128
4065644423573378304
4065644423573379712
4065644492293160960
4065644423573398144
4065644423573398656
4065644496562126976
4065644423573399296
4065644423573400320
4065644423573401472
4065644423573401856
4065644423573402880
4065644462202474752
4065644423573404928
4065644423573530880
4065644423573446272
4065644423573450880
4065644423573453312
4065644423573455616
4065644423573456768
4065644423573457792
4065644423573537280
4065644423573538048
4065644423573579392
4065644427842717952
4065644496564063104
4065644427842749440
4065644427917794048


4065644565356750848
4065644565356753152
4065644565356753536
4065644634001342592
4065644595373639296
4065644595373661952
4065644565356756352
4065644565356756992
4065644565356758016
4065644565356758144
4065644565356758912
4065644565356759552
4065644565356760704
4065644565356761216
4065644565356763008
4065644565356765184
4065644565356767360
4065644565356767616
4065644565356769920
4065644565356770048
4065644565356770432
4065644565356771200
4065644565356772352
4065644565356772608
4065644565356773504
4065644565356774656
4065644565356778880
4065644565356781824
4065644565356781952
4065644565356784640
4065644565356785664
4065644565356786304
4065644565356786432
4065644565356789248
4065644565356789632
4065644565357197824
4065644565357198976
4065644565357537408
4065644565357637376
4065644565357638144
4065644565357638400
4065644565357680896
4065644565357806592
4065644565357808384
4065644565357949184
4065644565377138304
4065644565377140480
4065644565377151232
4065644565377178112
4065644565377178240


...

4065633226639061248
4065633226639062144
4065633226639064832
4065633226639066496
4065633226639068800
4065633226639072256
4065633226639074944
4065633226639075456
4065633226639075840
4065633226639075968
4065633226639079296
4065633226639079680
4065633226639080576
4065633226639082496
4065633226639085056
4065633226639086976
4065633226639087232
4065633226639088128
4065633226639089792
4065633226639090176
4065633226639091584
4065633226639092480
4065633226639119232
4065633226639122560
4065633226639093248
4065633226639096064
4065633226639098752
4065633226639103104
4065633226639103488
4065633226639106688
4065633226639108096
4065633226639109376
4065633226639110400
4065633226639110528
4065633226639111552
4065633226639113216
4065633226639113728
4065633226639116416
4065633226639117568
4065633226639327104
4065633226639327744
4065633226639327872
4065633226639328384
4065633226639333504
4065633226639409792
4065633226639409920
4065633226639410048
4065633226640074624
4065633226640075136
4065633226640136832


1819859698712779648
1819859698712781568
1819859698712782464
1819859698712785920
1819859698712786944
1819859698712790656
1819859698712794496
1819859698713286528
1819859698713331456
1819859728735543680
1819859728735546112
1819859728735546624
1819859728735546752
1819859728735546880
1819859728736039680
1819859728749450752
1819859728757197312
1819859728757197440
1819859728757197952
1819859728757999872
1819859728758000512
1819859728758002560
1819859733053479424
1819859733053501312
1819859733053507584
1819859733053519744
1819859733053521024
1819859733053523712
1819859733053523840
1819859733053526400
1819859733053529856
1819859733053534080
1819859733053535488
1819859733053541248
1819859733053541760
1819859733053542144
1819859733053542400
1819859733053542528
1819859733053542656
1819859733053548928
1819859733053549056
1819859733053549952
1819859733053552512
1819859733053553024
1819859733053558528
1819859733053559168
1819859733053559936
1819859733053562624
1819859733053565056
1819859733053568640


1819860695145222528
1819860695145224960
1819860695145227392
1819860695145237760
1819860695145239168
1819860695145240064
1819860695145240192
1819860695145244544
1819860695145244672
1819860695145670912
1819860695148487552
1819860695148495744
1819860695148496640
1819860725168002816
1819860725168010496
1819860725168012416
1819860725168014336
1819860725168023424
1819860725168025600
1819860725168456960
1819860725168457728
1819860725172755456
1819860725172755840
1819860725181849344
1819860725181852288
1819860725189613568
1819860725189613696
1819860725189616512
1819860725189616640
1819860725189617920
1819860725189638016
1819860729486192128
1819860729486203136
1819860729486209792
1819860729486211328
1819860729486212096
1819860729486214912
1819860729486217984
1819860729486221568
1819860729486222080
1819860729486224512
1819860729486231168
1819860729486231680
1819860729486238208
1819860729486238336
1819860729486238464
1819860729486240768
1819860729486246656
1819860729486246784
1819860729486255488


1819861481096160896
1819861481104763776
1819861481104764928
1819861481104765184
1819861481104769280
1819861485400002688
1819861485400011136
1819861485400019840
1819861485400033536
1819861485400039040
1819861485400039168
1819861485400039296
1819861485400040960
1819861485400041088
1819861485400044032
1819861485400044544
1819861485400046208
1819861485400046592
1819861485400051968
1819861485400054144
1819861485400054912
1819861485400056960
1819861485400061696
1819861485400062976
1819861485400063104
1819861485400066816
1819861485400071936
1819861485400074368
1819861485400075392
1819861485400076416
1819861485400080512
1819861485400085760
1819861485400087936
1819861485400094080
1819861485400094848
1819861485400099456
1819861485400100992
1819861485400103808
1819861485400115712
1819861485403546368
1819861485403554560
1819861485403556736
1819861485403559168
1819861485403569664
1819861485403573248
1819861485403595520
1819861485403612288
1819861485403624448
1819861485403626880
1819861485405456000


1819865539855037696
1819865471130388864
1819865466811976960
1819865466811977088
1819865466811983872
1819865466811988864
1819865466811990016
1819865466811991168
1819865466811997056
1819865466812002688
1819865466812359680
1819865466825734272
1819865466833537408
1819865466833538176
1819865466833561984
1819865466833562112
1819865466833562368
1819865466833562496
1819865471130331264
1819865471130338176
1819865471130338304
1819865471130358784
1819865471130360832
1819865471130364672
1819865471130374912
1819865471130375680
1819865471130380800
1819865471130384768
1819865471130388992
1819865471130409216
1819865471130409344
1819865471130409856
1819865471130413568
1819865471130414592
1819865471130415616
1819865471130417152
1819865471130433024
1819865471133859456
1819865471133861760
1819865471133862272
1819865471133876096
1819865471133884672
1819865471133895552
1819865471133896448
1819865471133920768
1819865471133942784
1819865471133946496
1819865471133946624
1819865471135172352
1819865471135553664
